In [1]:
import numpy as np
import cv2
import os, sys
import time
import operator
from string import ascii_uppercase
import tkinter as tk
from PIL import Image, ImageTk
from hunspell import Hunspell
import enchant
from keras.models import model_from_json
from gtts import gTTS

os.environ["THEANO_FLAGS"] = "device=cuda, assert_no_cpu_op=True"

class Application:

    def __init__(self):
        self.hs = Hunspell('en_US')
        self.vs = cv2.VideoCapture(0)
        self.current_image = None
        self.current_image2 = None
        self.json_file = open("model-bw.json", "r")
        self.model_json = self.json_file.read()
        self.json_file.close()
        self.loaded_model = model_from_json(self.model_json)
        self.loaded_model.load_weights("model-bw.h5")

        self.ct = {}
        self.ct['blank'] = 0
        self.blank_flag = 0

        for i in ascii_uppercase:
            self.ct[i] = 0
        print("Loaded model from disk")
        self.root = tk.Tk()
        self.root.title("Sign Language To Text Conversion")
        self.root.protocol('WM_DELETE_WINDOW', self.destructor)
        self.root.geometry("1200x700")
        self.root.configure(bg='black')
        self.panel = tk.Label(self.root)
        self.panel.place(x=50, y=50, width=580, height=580)
        self.panel.config(bg="black")

        self.panel2 = tk.Label(self.root)  # initialize image panel
        self.panel2.place(x=350, y=105, width=275, height=275)
        

        self.T = tk.Label(self.root)
        self.T.place(x=380, y=12)
        self.T.config(text="DEAF & DUMB HELPER",fg = "white",bg="black" ,font=("Cancun", 35, "bold"))

        self.panel3 = tk.Label(self.root)  # Current Symbol
        self.panel3.place(x=935, y=120)
        self.T1 = tk.Label(self.root)
        self.T1.place(x=640, y=115)
        self.T1.config(text="Character :",fg = "white",bg="black", font=("Courier", 30, "bold"))

        
        self.panel4 = tk.Label(self.root)  # Word
        self.panel4.place(x=855, y=295)
        self.T2 = tk.Label(self.root)
        self.T2.place(x=640, y=290)
        self.T2.config(text="Word :",fg = "white",bg="black",  font=("Courier", 30, "bold"))
        self.bt4 = tk.Button(self.root, command= self.clear, height= 0,width=0)
        self.bt4.place (x=1200 , y = 290 )
        self.bt4.config(text= "Clear" ,fg = "white",bg="black",  font=("Courier", 20, "bold"))
        self.bt7 = tk.Button(self.root, command= self.voice1, height= 0,width=0)
        self.bt7.place (x=1310 , y = 290 )
        self.bt7.config(text= "Voice" ,fg = "white",bg="black",  font=("Courier", 20, "bold"))

        
        self.panel5 = tk.Label(self.root)  # Sentence
        self.panel5.place(x=855, y=505)
        
        self.T3 = tk.Label(self.root)
        self.T3.place(x=640, y=500)
        self.T3.config(text="Sentence :",fg = "white",bg="black",  font=("Courier", 30, "bold"))
        
        self.bt5 = tk.Button(self.root, command= self.clear1, height= 0,width=0)
        self.bt5.place (x=1200 , y = 500 )
        self.bt5.config(text= "Clear" ,fg = "white",bg="black",  font=("Courier", 20, "bold"))
        
        self.bt6 = tk.Button(self.root, command= self.voice, height= 0,width=0)
        self.bt6.place (x=1310 , y = 500 )
        self.bt6.config(text= "Voice" ,fg = "white",bg="black",  font=("Courier", 20, "bold"))

        
        self.T4 = tk.Label(self.root)
        self.T4.place(x=350, y=655)
        self.T4.config(text="Suggestions :", fg = "white",bg="black",  font=("Courier", 30, "bold"))
        self.bt1 = tk.Button(self.root, command=self.action1, height=0, width=0)
        self.bt1.place(x=700, y=660)
        self.bt2 = tk.Button(self.root, command=self.action2, height=0, width=0)
        self.bt2.place(x=920, y=660)
        self.bt3 = tk.Button(self.root, command=self.action3, height=0, width=0)
        self.bt3.place(x=1140, y=660)
        
        

        self.str = ""
        self.word = ""
        self.current_symbol = "Empty"
        self.photo = "Empty"
        self.video_loop()

    def video_loop(self):
        ok, frame = self.vs.read()

        if ok:
            cv2image = cv2.flip(frame, 1)

            x1 = int(0.5 * frame.shape[1])
            y1 = 10
            x2 = frame.shape[1] - 10
            y2 = int(0.5 * frame.shape[1])

            cv2.rectangle(frame, (x1 - 1, y1 - 1), (x2 + 1, y2 + 1), (255, 0, 0), 1)
            cv2image = cv2.cvtColor(cv2image, cv2.COLOR_BGR2RGBA)

            self.current_image = Image.fromarray(cv2image)
            imgtk = ImageTk.PhotoImage(image=self.current_image)

            self.panel.imgtk = imgtk
            self.panel.config(image=imgtk)

            cv2image = cv2image[y1: y2, x1: x2]

            gray = cv2.cvtColor(cv2image, cv2.COLOR_BGR2GRAY)

            blur = cv2.GaussianBlur(gray, (5, 5), 2)

            th3 = cv2.adaptiveThreshold(blur, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)

            ret, res = cv2.threshold(th3, 70, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)

            self.predict(res)

            self.current_image2 = Image.fromarray(res)

            imgtk = ImageTk.PhotoImage(image=self.current_image2)

            self.panel2.imgtk = imgtk
            self.panel2.config(image=imgtk)

            self.panel3.config(text=self.current_symbol,fg = "white",bg="black",  font=("Courier", 25))

            self.panel4.config(text=self.word,fg = "white",bg="black",  font=("Courier", 25))

            self.panel5.config(text=self.str,fg = "white",bg="black",  font=("Courier", 25))

            predicts = self.hs.suggest(self.word)

            if (len(predicts) > 1):

                self.bt1.config(text=predicts[0],fg = "white",bg="black",  font=("Courier", 20))

            else:

                self.bt1.config(text="")

            if (len(predicts) > 2):

                self.bt2.config(text=predicts[1],fg = "white",bg="black",  font=("Courier", 20))

            else:

                self.bt2.config(text="")

            if (len(predicts) > 3):

                self.bt3.config(text=predicts[2],fg = "white",bg="black",  font=("Courier", 20))

            else:

                self.bt3.config(text="")

        self.root.after(5, self.video_loop)

    def predict(self, test_image):

        test_image = cv2.resize(test_image, (128, 128))
        result = self.loaded_model.predict(test_image.reshape( 1,128, 128, 1))
        
        categories = {0: 'blank', 1: 'A', 2: 'B', 3: 'C', 4: 'D', 5: 'E', 6: 'F', 7: 'G', 8: 'H',
              9: 'I', 10: 'J', 11: 'K', 12: 'L', 13: 'M',
              14: 'N', 15: 'O', 16: 'P', 17: 'Q', 18: 'R', 19: 'S', 20: 'T', 21: 'U', 22: 'V', 23: 'W', 24: 'X', 25: 'Y', 26: 'Z'}


        prediction = {'blank': result[0][0], 
                      'A': result[0][1], 
                      'B': result[0][2],
                      'C': result[0][3],
                      'D': result[0][4],
                      'E': result[0][5],
                      'F': result[0][6],
                      'G': result[0][7],
                      'H': result[0][8],
                      'I': result[0][9],
                      'J': result[0][10],
                      'K': result[0][11],
                      'L': result[0][12],
                      'M': result[0][13],
                      'N': result[0][14],
                      'O': result[0][15],
                      'P': result[0][16],
                      'Q': result[0][17],
                      'R': result[0][18],
                      'S': result[0][19],
                      'T': result[0][20],
                      'U': result[0][21],
                      'V': result[0][22],
                      'W': result[0][23],
                      'X': result[0][24],
                      'Y': result[0][25],
                      'Z': result[0][26]}
        prediction = sorted(prediction.items(), key=operator.itemgetter(1), reverse=True)
        self.current_symbol = prediction[0][0]
        if (self.current_symbol == 'blank'):
            for i in ascii_uppercase:
                self.ct[i] = 0
        self.ct[self.current_symbol] += 1
        if (self.ct[self.current_symbol] > 60):
            for i in ascii_uppercase:
                if i == self.current_symbol:
                    continue
                tmp = self.ct[self.current_symbol] - self.ct[i]
                if tmp < 0:
                    tmp *= -1
                if tmp <= 20:
                    self.ct['blank'] = 0
                    for i in ascii_uppercase:
                        self.ct[i] = 0
                    return
            self.ct['blank'] = 0
            for i in ascii_uppercase:
                self.ct[i] = 0
            if self.current_symbol == 'blank':
                if self.blank_flag == 0:
                    self.blank_flag = 1
                    if len(self.str) > 0:
                        self.str += " "
                    self.str += self.word
                    self.word = ""
            else:
                if (len(self.str) > 16):
                    self.str = ""
                self.blank_flag = 0
                self.word += self.current_symbol  
    def clear(self):
        self.word = ""
        
    def clear1(self):
        self.str = ""
        
    def voice(self):
        language = 'en'
        myobj = gTTS(text=self.str, lang=language, slow=False)
        # welcome
        myobj.save("Voice.mp3")
        # Playing the converted file
        os.system("start Voice.mp3")
        
    def voice1(self):
        language = 'en'
        myobj = gTTS(text=self.word, lang=language, slow=False)
        # welcome
        myobj.save("Voice1.mp3")
        # Playing the converted file
        os.system("start Voice1.mp3")
    
    def action1(self):
        predicts = self.hs.suggest(self.word)
        if (len(predicts) > 0):
            self.word = ""
            self.str += " "
            self.str += predicts[0]
    def action2(self):
        predicts = self.hs.suggest(self.word)
        if (len(predicts) > 1):
            self.word = ""
            self.str += " "
            self.str += predicts[1]
    def action3(self):
        predicts = self.hs.suggest(self.word)
        if (len(predicts) > 2):
            self.word = ""
            self.str += " "
            self.str += predicts[2]
    def destructor(self):
        print("Closing Application...")
        self.root.destroy()
        self.vs.release()
        cv2.destroyAllWindows()
print("Starting Application...")
(Application()).root.mainloop()

Starting Application...
Loaded model from disk
Closing Application...
